Importing required libraries

In [ ]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
import re
from nltk.corpus import stopwords
from google.colab import drive

In [ ]:
a3 = np.array([[1,2,3,4], [5,6,7,8], [9,10,11,12]])
a3

array([[ 1,  2,  3,  4],
       [ 5,  6,  7,  8],
       [ 9, 10, 11, 12]])

In [ ]:
d3 = np.random.randn(3,4) < 0.7
print(d3)

[[ True  True  True False]
 [ True False  True False]
 [False  True False  True]]


In [ ]:
print(np.multiply(a3, d3))

[[ 1  2  3  0]
 [ 5  0  7  0]
 [ 0 10  0 12]]


In [ ]:
print(np.multiply(a3, d3)/0.7)

[[ 1.42857143  2.85714286  4.28571429  0.        ]
 [ 7.14285714  0.         10.          0.        ]
 [ 0.         14.28571429  0.         17.14285714]]


Reading the file:

In [ ]:
drive.mount('/content/drive/')
df = pd.read_csv(r"/content/drive/MyDrive/Train dataset.csv",sep=',')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


Preprocessing to remove punctuations and stopwords:

In [ ]:
import nltk
nltk.download('stopwords')
stop_words = stopwords.words('english')
df['review'] = df['review'].str.lower()
df['review'] = df['review'].str.replace(r'[^\w\s]+', '')
df['review'] = df['review'].apply(lambda x:' '.join([word for word in x.split() if word not in (stop_words)]))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  """


In [ ]:
tokenizer = Tokenizer(num_words=10000)

In [ ]:
tokenizer.fit_on_texts(df['review'].values)
X = tokenizer.texts_to_sequences(df['review'])

In [ ]:
from sklearn.preprocessing import LabelEncoder
Le = LabelEncoder()
y = Le.fit_transform(df['sentiment'])

In [ ]:
#Hyperparametes
embed_dim = 16
lstm_out = 64
val_split = 0.1
epoches = 4
batch_sizes = 128
#---------------------------
max_input_len = 2000
vocabSize = 13000
# for i in range(len(df.index)):
#   max_input_len = max(max_input_len,len(df['review'][i]))
#   vocabSize = max(vocabSize,max(X[i]))

#Making LSTM model
model = Sequential()
model.add(Embedding(vocabSize, embed_dim, input_length = max_input_len))
model.add(LSTM(lstm_out, dropout=0.2))   # recurrent_dropout=0.2))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss = 'binary_crossentropy', optimizer='adam',metrics = ['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 2000, 16)          208000    
                                                                 
 lstm (LSTM)                 (None, 64)                20736     
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 228,801
Trainable params: 228,801
Non-trainable params: 0
_________________________________________________________________


In [ ]:
X = pad_sequences(X, maxlen=max_input_len)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X,y, test_size = val_split)

In [ ]:
model.fit(X_train, y_train, validation_data = (X_val,y_val), epochs = epoches, batch_size=batch_sizes)

Epoch 1/4
282/282 [==============================] - 27s 85ms/step - loss: 0.4250 - accuracy: 0.7979 - val_loss: 0.2751 - val_accuracy: 0.8865
Epoch 2/4
282/282 [==============================] - 24s 84ms/step - loss: 0.2244 - accuracy: 0.9142 - val_loss: 0.2635 - val_accuracy: 0.8985
Epoch 3/4
282/282 [==============================] - 31s 111ms/step - loss: 0.1842 - accuracy: 0.9305 - val_loss: 0.2656 - val_accuracy: 0.8978
Epoch 4/4
282/282 [==============================] - 24s 84ms/step - loss: 0.2353 - accuracy: 0.9029 - val_loss: 0.3078 - val_accuracy: 0.8867


In [ ]:
df1 = pd.read_csv(r"/content/drive/MyDrive/Test Dataset.csv",sep=',')

In [ ]:
df1.head(5)

,review,sentiment
0,Recap: Not entirely familiar with the Shakespe...,negative
1,"It has been 16 years since it's original run, ...",positive
2,I'm an animator myself and an all around buff ...,negative
3,The movie had no excitement and does not have ...,negative
4,i just got puzzled why damn FOX canceled the s...,positive


In [ ]:
df1['review'] = df1['review'].str.lower()
df1['review'] = df1['review'].str.replace(r'[^\w\s]+', '')
df1['review'] = df1['review'].apply(lambda x:' '.join([word for word in x.split() if word not in (stop_words)]))
# tokenizer.fit_on_texts(df1['review'].values)
X1 = tokenizer.texts_to_sequences(df1['review'])
y1 = Le.fit_transform(df1['sentiment'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  


In [ ]:
X1 = pad_sequences(X1, maxlen=max_input_len)
model.evaluate(X1,y1)

313/313 [==============================] - 7s 23ms/step - loss: 0.3211 - accuracy: 0.8808


[0.32107144594192505, 0.8808000087738037]